### Reverse mode autodiff

In [13]:
from typing import List
from collections import defaultdict
import numpy as np

1. Make computational graph
2. Use relations to generate adjoints
3. Test on a few cases

In [ ]:
class Variable:
    def __init__(self, value, local_gradients=[]):
        self.value = value
        self.local_gradients = local_gradients
    
    def __add__(self, other):
        return add(self, other)
    
    def __radd__(self, other):
        self.value += other
        return self
    
    def __mul__(self, other):
        return mul(self, other)
    
    def __rmul__(self, other):
        self.value *= other
        self.local_gradients
    
    def __sub__(self, other):
        return add(self, neg(other))
    
    def __rsub__(self, other):
        return add(self, -other)

    def __truediv__(self, other):
        return mul(self, inv(other))
    
    def __rtruediv__(self, other):
        return mul(self, 1 / other)
    
    def __str__(self):
        return f"Value {self.value}"
    
def add(a, b):
    value = a.value + b.value    
    local_gradients = (
        (a, 1),
        (b, 1)
    )
    return Variable(value, local_gradients)


def mul(a, b):
    value = a.value * b.value    
    local_gradients = (
        (a, b.value),
        (b, a.value)
    )
    return Variable(value, local_gradients)

def neg(a):
    value = -1 * a.value
    local_gradients = (
        (a, -1),
    )
    return Variable(value, local_gradients)

def inv(a):
    value = 1. / a.value
    local_gradients = (
        (a, -1 / a.value**2),
    )
    return Variable(value, local_gradients)     

In [17]:
def sin(a):
    value = np.sin(a.value)
    local_gradients = (
        (a, np.cos(a.value)),
    )
    return Variable(value, local_gradients)

def exp(a):
    value = np.exp(a.value)
    local_gradients = (
        (a, value),
    )
    return Variable(value, local_gradients)
    
def log(a):
    value = np.log(a.value)
    local_gradients = (
        (a, 1. / a.value),
    )
    return Variable(value, local_gradients)

In [18]:
def get_gradients(variable):
    """ Compute the first derivatives of `variable` 
    with respect to child variables.
    """
    gradients = defaultdict(lambda: 0)
    
    def compute_gradients(variable, path_value):
        for child_variable, local_gradient in variable.local_gradients:
            # "Multiply the edges of a path":
            value_of_path_to_child = path_value * local_gradient
            # "Add together the different paths":
            gradients[child_variable] += value_of_path_to_child
            # recurse through graph:
            compute_gradients(child_variable, value_of_path_to_child)
    
    compute_gradients(variable, path_value=1)
    # (path_value=1 is from `variable` differentiated w.r.t. itself)
    return gradients

In [19]:
print(a * b)

TypeError: unsupported operand type(s) for *: 'float' and 'Variable'

In [ ]:
def f(a, b, c):
    return a*b + b*c


a = Variable(230.3)
b = Variable(33.2)
c = Variable(23.3)
y = f(a, b, c)

gradients = get_gradients(y)

print("The partial derivative of y with respect to a =", gradients[a])
print("The partial derivative of y with respect to b =", gradients[b])
print(gradients[c])

TypeError: unsupported operand type(s) for +: 'NoneType' and 'NoneType'

In [81]:
gradients

defaultdict(<function __main__.get_gradients.<locals>.<lambda>()>,
            {<__main__.Variable at 0x1f0fe9327b0>: 1,
             <__main__.Variable at 0x1f0ec628b90>: 33.2,
             <__main__.Variable at 0x1f0fe933350>: 253.60000000000002,
             <__main__.Variable at 0x1f0fe933770>: 1,
             <__main__.Variable at 0x1f0fe932f00>: 33.2})